<h3> Data Analyst Spotify Case Study | SQL Interview Questions </h3>

In [0]:
%sql

CREATE table activity
(
user_id varchar(20),
event_name varchar(20),
event_date date,
country varchar(20)
);
delete from activity;
insert into activity values (1,'app-installed','2022-01-01','India')
,(1,'app-purchase','2022-01-02','India')
,(2,'app-installed','2022-01-01','USA')
,(3,'app-installed','2022-01-01','USA')
,(3,'app-purchase','2022-01-03','USA')
,(4,'app-installed','2022-01-03','India')
,(4,'app-purchase','2022-01-03','India')
,(5,'app-installed','2022-01-03','SL')
,(5,'app-purchase','2022-01-03','SL')
,(6,'app-installed','2022-01-04','Pakistan')
,(6,'app-purchase','2022-01-04','Pakistan');

num_affected_rows,num_inserted_rows
11,11


In [0]:
%sql
select * from activity

user_id,event_name,event_date,country
1,app-installed,2022-01-01,India
1,app-purchase,2022-01-02,India
2,app-installed,2022-01-01,USA
3,app-installed,2022-01-01,USA
3,app-purchase,2022-01-03,USA
4,app-installed,2022-01-03,India
4,app-purchase,2022-01-03,India
5,app-installed,2022-01-03,SL
5,app-purchase,2022-01-03,SL
6,app-installed,2022-01-04,Pakistan


<h3> Solution in sql(hive) </h3>

<h3> Daily active users <\h3>

In [0]:
%sql 
select event_date,count( distinct user_id) as daily_user from activity group by event_date

event_date,daily_user
2022-01-03,3
2022-01-04,1
2022-01-01,3
2022-01-02,1


<h3> Weekly active users <\h3>

In [0]:
%sql
select case when weekofyear(event_date)=52 then 1 else weekofyear(event_date)+1 end as week,count(distinct user_id)  
from activity
group by weekofyear(event_date)

week,count(DISTINCT user_id)
2,4
1,3


<h3> same day install and purchase <\h3>

In [0]:
%sql
SELECT e1.event_date,COALESCE(e2.cnt,0) FROM (SELECT DISTINCT event_date FROM activity)e1
left join (
SELECT a1.event_date,count(distinct a1.user_id)cnt FROM activity a1
INNER JOIN activity a2 on a1.user_id=a2.user_id and a1.event_date=a2.event_date
WHERE a1.event_name='app-installed' and a2.event_name='app-purchase'
GROUP BY a1.event_date)e2 on e2.event_date=e1.event_date;


event_date,"coalesce(cnt, 0)"
2022-01-03,2
2022-01-04,1
2022-01-01,0
2022-01-02,0


<h3> country wise paid users <\h3>

In [0]:
%sql
with cte as(
select case when country in ('India','Pakistan') then country else 'others' end as country,count(distinct user_id) cnt
 from activity where event_name='app-purchase'
 group by case when country in ('India','Pakistan') then country else 'others' end)

 select country,(cnt/(select sum(cnt) from cte))*100 as percentage from cte

country,percentage
India,40.0
others,40.0
Pakistan,20.0


<h3> install and purchase on next day <\h3

In [0]:
%sql
SELECT e1.event_date,COALESCE(e2.cnt,0) FROM (SELECT DISTINCT event_date FROM activity)e1
left join (
SELECT a2.event_date,count(DISTINCT a2.user_id)cnt FROM activity a1
INNER JOIN activity a2 on a1.user_id=a2.user_id and date_diff(a2.event_date,a1.event_date)=1
WHERE a1.event_name='app-installed' and a2.event_name='app-purchase'
GROUP BY a2.event_date)e2 on e2.event_date=e1.event_date;



event_date,"coalesce(cnt, 0)"
2022-01-03,0
2022-01-04,0
2022-01-01,0
2022-01-02,1


<h3> solution in pyspark <\h3>

In [0]:
from pyspark.sql.functions import col,count,weekofyear,when,lit,sum,datediff

In [0]:
df_a=spark.sql("select * from activity")
df_a.show()

+-------+-------------+----------+--------+
|user_id|   event_name|event_date| country|
+-------+-------------+----------+--------+
|      1|app-installed|2022-01-01|   India|
|      1| app-purchase|2022-01-02|   India|
|      2|app-installed|2022-01-01|     USA|
|      3|app-installed|2022-01-01|     USA|
|      3| app-purchase|2022-01-03|     USA|
|      4|app-installed|2022-01-03|   India|
|      4| app-purchase|2022-01-03|   India|
|      5|app-installed|2022-01-03|      SL|
|      5| app-purchase|2022-01-03|      SL|
|      6|app-installed|2022-01-04|Pakistan|
|      6| app-purchase|2022-01-04|Pakistan|
+-------+-------------+----------+--------+



<h3> Daily active users <\h3>

In [0]:
df_a.select("event_date","user_id").distinct().groupBy("event_date").count().orderBy("event_date").show()

+----------+-----+
|event_date|count|
+----------+-----+
|2022-01-01|    3|
|2022-01-02|    1|
|2022-01-03|    3|
|2022-01-04|    1|
+----------+-----+



<h3> Weekly active users <\h3>

In [0]:
df_a1=df_a.select("event_date","user_id").distinct().withColumn("weekno",weekofyear(df_a["event_date"]))
df_a1=df_a1.withColumn("week_no",when(col("weekno")==52,1).otherwise(col("weekno")+1))
df_a1.select("week_no","user_id").distinct().groupBy("week_no").count().show()

+-------+-----+
|week_no|count|
+-------+-----+
|      1|    3|
|      2|    4|
+-------+-----+



<h3> same day install and purchase <\h3>

In [0]:
df1=df_a.filter(col("event_name")=="app-installed").select("user_id",col("event_date").alias("event"))
df2=df_a.filter(col("event_name")!="app-installed").select("user_id","event_date")
df_f=df2.join(df1,(df1.user_id==df2.user_id)&(df1.event==df2.event_date),"inner").\
    select(df2["user_id"],df2["event_date"]).distinct().groupBy("event_date").count()
df_a.join(df_f,"event_date","left").select("event_date","count").distinct().fillna(0).orderBy("event_date").show()

+----------+-----+
|event_date|count|
+----------+-----+
|2022-01-01|    0|
|2022-01-02|    0|
|2022-01-03|    2|
|2022-01-04|    1|
+----------+-----+



<h3> country wise paid users <\h3>

In [0]:
df_a1=df_a.filter(col("event_name")=="app-purchase").withColumn("country",when(col("country")\
    .isin("India","USA"),col("country")).otherwise("other")).select("country","user_id").distinct()\
        .groupBy("country").count()
df=df_a1.agg(sum(col("count"))).collect()[0][0]
df_a1.withColumn("percentage",col("count")*100/df).select("country","percentage").show()

+-------+----------+
|country|percentage|
+-------+----------+
|  India|      40.0|
|  other|      40.0|
|    USA|      20.0|
+-------+----------+



<h3> install and purchase on next day <\h3

In [0]:
df1=df_a.filter(col("event_name")=="app-installed").select("user_id",col("event_date").alias("event"))
df2=df_a.filter(col("event_name")!="app-installed").select("user_id","event_date")
df_f=df2.join(df1,(df1.user_id==df2.user_id)&(datediff(df2.event_date,df1.event)==1),"inner").\
    select(df2["user_id"],df2["event_date"]).distinct().groupBy("event_date").count()
df_a.join(df_f,"event_date","left").select("event_date","count").distinct().fillna(0).orderBy("event_date").show()

+----------+-----+
|event_date|count|
+----------+-----+
|2022-01-01|    0|
|2022-01-02|    1|
|2022-01-03|    0|
|2022-01-04|    0|
+----------+-----+

